In [1]:
class treeNode:
    def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue       #名字变量
        self.count = numOccur       #计数变量（频率）
        self.nodeLink = None        #链接相似元素项
        self.parent = parentNode    #当前父节点
        self.children = {}          #用于存放子节点
    
    def inc(self, numOccur):
        self.count += numOccur
        
    def disp(self, ind=1):
        print('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)         #子节点向右缩减

In [2]:
def loadSimpDat():
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [3]:
def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        fset = frozenset(trans)
        retDict.setdefault(fset, 0)
        retDict[fset] += 1
    return retDict

In [4]:
def updateHeader(nodeToTest, targetNode):  
    while (nodeToTest.nodeLink != None):  
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

In [5]:
def updateTree(items, myTree, headerTable, count):
    if items[0] in myTree.children:  
        myTree.children[items[0]].inc(count) 
    else:  
        myTree.children[items[0]] = treeNode(items[0], count, myTree)
        if headerTable[items[0]][1] == None: 
            headerTable[items[0]][1] = myTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1], myTree.children[items[0]])
    if len(items) > 1:  
        updateTree(items[1:], myTree.children[items[0]], headerTable, count)

In [6]:
def createTree(dataSet, minSup=1):
    headerTable = {}
    #第一次遍历数据集， 记录每个数据项的支持度
    for trans in dataSet:
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + 1
    
    #根据最小支持度过滤
    lessThanMinsup = list(filter(lambda k:headerTable[k] < minSup, headerTable.keys()))
    for k in lessThanMinsup: 
        del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    
    #如果所有数据都不满足最小支持度，返回None, None
    if len(freqItemSet) == 0:
        return None, None
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    myTree = treeNode('φ', 1, None)
   
   #第二次遍历数据集，构建fp-tree
    for tranSet, count in dataSet.items():
        #根据最小支持度处理一条训练样本，key:样本中的一个样例，value:该样例的的全局支持度
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            #根据全局频繁项对每个事务中的数据进行排序,等价于 order by p[1] desc, p[0] desc
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: (p[1],p[0]), reverse=True)]
            updateTree(orderedItems, myTree, headerTable, count)
    return myTree, headerTable

In [7]:
def ascendTree(leafNode, prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent, prefixPath)

In [8]:
def findPrefixPath(basePat, headerTable):
    condPats = {}
    treeNode = headerTable[basePat][1]
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode, prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [9]:
def mineTree(inTree, headerTable, minSup=1, preFix=set([]), freqItemList=[]):
    #排序minSup asc, value asc
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: (p[1][0],p[0]))]
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        # 通过条件模式基找到的频繁项集
        condPattBases = findPrefixPath(basePat, headerTable)
        myCondTree, myHead = createTree(condPattBases, minSup)
        if myHead != None:
            print('condPattBases: ', basePat, condPattBases)
            myCondTree.disp()
            print('*' * 30)
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

In [10]:
simpDat = loadSimpDat()
dictDat = createInitSet(simpDat)
myFPTree,myheader = createTree(dictDat, 3)
myFPTree.disp()

   φ   1
     z   5
       r   1
       x   3
         y   3
           t   3
             s   2
             r   1
     x   1
       s   1
         r   1


In [15]:
mineTree(myFPTree, myheader, 2)

condPattBases:  r {frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'t', 'z', 'y', 'x'}): 1}
   φ   1
     z   2
       x   1
     x   1
******************************
condPattBases:  s {frozenset({'t', 'z', 'y', 'x'}): 2, frozenset({'x'}): 1}
   φ   1
     x   3
******************************
